# Install Requirements

# Test

In [2]:
import torch
import torch.nn as nn
from transformers import AutoConfig, ElectraForMaskedLM, AutoTokenizer
from ElectraKAN import ElectraGenerator


torch.manual_seed(42)

generator_tokenizer = AutoTokenizer.from_pretrained('google/electra-base-generator')
generator_config = AutoConfig.from_pretrained('google/electra-base-generator')


discriminator_tokenizer = AutoTokenizer.from_pretrained('google/electra-base-discriminator')
discriminator_config = AutoConfig.from_pretrained('google/electra-base-discriminator')


random_input_ids = torch.randint(0, len(generator_tokenizer), (1, 512))
random_attention_mask = torch.randint(0, 1, (1, 512))
random_token_type_ids = torch.randint(0, 1, (1, 512))

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
orig_generator = ElectraForMaskedLM(generator_config)
generator = ElectraGenerator(
    vocab_size=len(generator_tokenizer),
    embedding_dim=768,
    vocab_type_size=2,
    layernorm_eps=1e-12,
    embedding_dropout_p=.1,
    hidden_dim=768,
    num_heads=4,
    ff_dim=1024,
    num_layers=12,
    max_pos_embedding=512
) 


print("initialzation complete.")

with torch.no_grad():
    orig_generator.eval()
    generator.eval()

    orig_output = orig_generator(input_ids=random_input_ids, attention_mask=random_attention_mask, token_type_ids=random_token_type_ids)
    output = generator(input_ids=random_input_ids, attention_mask=random_attention_mask, token_type_ids=random_token_type_ids)

initialzation complete.
dim: 2, size[1]: 768, in_feature: 768
dim: 2, size[1]: 768, in_feature: 768
dim: 2, size[1]: 768, in_feature: 768
dim: 2, size[1]: 1024, in_feature: 1024
dim: 2, size[1]: 768, in_feature: 768
dim: 2, size[1]: 768, in_feature: 768
dim: 2, size[1]: 768, in_feature: 768
dim: 2, size[1]: 1024, in_feature: 1024
dim: 2, size[1]: 768, in_feature: 768
dim: 2, size[1]: 768, in_feature: 768
dim: 2, size[1]: 768, in_feature: 768
dim: 2, size[1]: 1024, in_feature: 1024
dim: 2, size[1]: 768, in_feature: 768
dim: 2, size[1]: 768, in_feature: 768
dim: 2, size[1]: 768, in_feature: 768
dim: 2, size[1]: 1024, in_feature: 1024
dim: 2, size[1]: 768, in_feature: 768
dim: 2, size[1]: 768, in_feature: 768
dim: 2, size[1]: 768, in_feature: 768
dim: 2, size[1]: 1024, in_feature: 1024
dim: 2, size[1]: 768, in_feature: 768
dim: 2, size[1]: 768, in_feature: 768
dim: 2, size[1]: 768, in_feature: 768
dim: 2, size[1]: 1024, in_feature: 1024
dim: 2, size[1]: 768, in_feature: 768
dim: 2, size[1

In [15]:
torch.allclose(orig_output.logits, output)

False